##### Copyright 2019 The TensorFlow Authors.

In [ ]:
!unzip /content/exc.zip

In [6]:
!pip uninstall tensorflow -y
!pip install  tensorflow==2.15

Found existing installation: tensorflow 2.10.0
Uninstalling tensorflow-2.10.0:
  Successfully uninstalled tensorflow-2.10.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 89.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully un

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)

Num GPUs Available:  1
2.15.0


In [2]:
import os
from glob import glob
import cv2, os, gc
from sklearn.model_selection import train_test_split

root_path = "/content/exc/"

In [4]:
folder_names = sorted(os.listdir(root_path))

y_list = []
lista = []
x_list = []

for fol in folder_names[:]:
    left = glob(os.path.join(root_path, fol, '*'))
    imPaths = [path for path in left if '.jpg' in path]
    for imPath in imPaths:
        #image = cv2.imread(imPath, cv2.IMREAD_GRAYSCALE)
        image = cv2.imread(imPath)
        image = cv2.resize(image, (84,84), cv2.INTER_AREA) #reisezeamos al tamaño de entrada
        x_list.append(image)
        post_fol = str(fol[4:])# Hago un croping del name del folder para que no haya string y quede parte entera
        s = post_fol.replace("_", "")
        s = int(s)
        y_list.append(s)

    continue
print("FIN")

FIN


In [5]:
import numpy as np
a = np.array(y_list)
categories, inverse = np.unique(a, return_inverse=True)

y_list_y = np.zeros((a.size, categories.size))
y_list_y[np.arange(a.size), inverse] = 1   ## one hot encode

x_list_x = np.asarray(x_list) ## pasar x's a un array de numpy

In [6]:
x_list_x.shape

(264, 84, 84, 3)

In [7]:
y_list_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [8]:
y_list_y.shape

(264, 44)

### Acá comienza

### Importante

In [28]:
#Import Package
from __future__ import absolute_import, division, print_function, unicode_literals
#import tensorflow
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense,Reshape,Flatten,Conv2D,Conv2DTranspose,LeakyReLU,ReLU,GlobalAveragePooling1D
from tensorflow.keras.layers import BatchNormalization,Dropout,Embedding,Activation,Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
#from tensorflow.keras.preprocessing.image import image
#from keras.utils.np_utils import to_categorical
from tensorflow.keras import backend as K
K.clear_session()
#other library
import os
import numpy as np
import matplotlib.pyplot as plt
import io
from PIL import Image
import math
import datetime
import pandas as pd
import random
from tensorboard.plugins.hparams import api as hp
#installation
#!pip install tensorflow-addons==0.8.3
#import tensorflow_addons as tfa
from mlxtend.plotting import plot_confusion_matrix

In [29]:
#Data Preprocessing with augmentation

TRAINING_DIR = "/content/exc/" #Training Set Directory
training_datagen = ImageDataGenerator(rescale = 1./255,)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(84,84),
    class_mode='categorical',
    batch_size=16
)

Found 264 images belonging to 44 classes.


In [30]:
#Model 1
input_ = Input(shape=(84, 84,3))
pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(input_shape = (84, 84, 3),
                                input_tensor = input_,
                                include_top = False,
                                weights = None
                                )
for layer in pre_trained_model.layers:
    layer.trainable = True
inception_layer = pre_trained_model.get_layer('mixed10')
inception_layer = inception_layer.output
inception_layer = layers.Flatten()(inception_layer)
print('last layer output shape: ', inception_layer.shape)

last layer output shape:  (None, 2048)


In [31]:
#Model 2
pre_trained_model_1 = tf.keras.applications.resnet50.ResNet50(input_shape = (84, 84, 3),
                                input_tensor = input_,
                                include_top = False,
                                weights = None
                                )
for layer in pre_trained_model_1.layers:
    layer.trainable = True
resnet_layer = pre_trained_model_1.get_layer('conv5_block3_out')
#pre_trained_model_1.summary()
resnet_layer  = resnet_layer. output
#x2 = layers.Conv2D(2048,(4,4),activation='relu')(x2)
resnet_layer  = layers.Flatten()(resnet_layer )
print('last layer output shape: ', resnet_layer.shape)

last layer output shape:  (None, 18432)


In [34]:
#Vision Transformer
#Tuneble Prameters
weight_decay = 0.0001
num_epochs = 150
num_classes= 44         # Tiene que la cantidad de clases de las personas
image_size = 84   ### tiene que ser cuadrado
patch_size = 6
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]
transformer_layers = 8
mlp_head_units = [2048, 1024]

In [35]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [36]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1.0, self.patch_size, self.patch_size, 1.0],
            strides=[1.0, self.patch_size, self.patch_size, 1.0],
            rates=[1.0, 1.0, 1.0, 1.0],
            padding="VALID",
        )
        patch_dims = tf.shape(patches)[-1]
        #patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [37]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )


    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)

        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [38]:
patches = Patches(patch_size)(input_)
encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

for _ in range(transformer_layers):
    x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    attention_output = layers.MultiHeadAttention(
    num_heads=num_heads, key_dim=projection_dim, dropout=0.1
    )(x1, x1)
    x2 = layers.Add()([attention_output, encoded_patches])
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
    x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
    encoded_patches = layers.Add()([x3, x2])
representation = layers.LayerNormalization(name='last_layer',epsilon=1e-6)(encoded_patches)
representation = layers.Flatten()(representation)
representation = layers.Dropout(0.5)(representation) ## quiza cambair droputs
representation= mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
print('last layer output shape: ', representation.shape)

last layer output shape:  (None, 1024)


In [39]:
#Merge Model
last_layer = layers.Concatenate()([inception_layer , resnet_layer, representation])
x = layers.Dense(1024, activation='relu')(last_layer)
x = tf.keras.layers.Dropout(0.2)(x)
x = layers.BatchNormalization(name='batch_1')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = ReLU()(x)
x = Dense(512)(x)
x = layers.BatchNormalization(name='batch_2')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = ReLU()(x)
x = Dense(256)(x)
x = BatchNormalization(name='batch_3')(x)
x = ReLU()(x)
x = layers.Dense(44, activation='softmax')(x) ## Densiddad de la ultiam layer tien q ser la cantidad de PERSONAS (clases)
                                                #quizas camibar activacion, normalzaiciones


In [40]:
#Model Compile
model = Model(inputs=input_,outputs= x)
model.compile(optimizer = Adam(learning_rate=0.001), ## Quiza cmabir optimizer y el learning rate
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics = ['accuracy','Precision', 'Recall'])

In [41]:
#Model
history = model.fit(
                train_generator,
#             x = x_list_x,
#             y = y_list_y,
            epochs = 30,
            verbose = 1,
            )

Epoch 1/30
17/17 [==============================] - 98s 214ms/step - loss: 3.9152 - accuracy: 0.0455 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/30
17/17 [==============================] - 3s 162ms/step - loss: 3.5433 - accuracy: 0.0909 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/30
17/17 [==============================] - 3s 162ms/step - loss: 3.1548 - accuracy: 0.1591 - precision: 0.2857 - recall: 0.0076
Epoch 4/30
17/17 [==============================] - 3s 161ms/step - loss: 2.7807 - accuracy: 0.1856 - precision: 0.1818 - recall: 0.0076
Epoch 5/30
17/17 [==============================] - 3s 187ms/step - loss: 2.5087 - accuracy: 0.2727 - precision: 0.6471 - recall: 0.0417
Epoch 6/30
17/17 [==============================] - 3s 163ms/step - loss: 2.2530 - accuracy: 0.3068 - precision: 0.5758 - recall: 0.0720
Epoch 7/30
17/17 [==============================] - 3s 163ms/step - loss: 2.1984 - accuracy: 0.3939 - precision: 0.7353 - recall: 0.0947
Epoch 8/30
17/17 [======

## RESOLUCION

Se entreno el modelo con todos los dedos, obteniendo como parte de validacion del entrenamiento altos niveles de accuracy y precision

In [54]:
prediction = model.predict(
    train_generator, verbose='auto', steps=None, callbacks=None,
)

17/17 [==============================] - 6s 59ms/step


In [57]:
prediction

array([[1.8838522e-04, 3.6817003e-04, 7.8980130e-04, ..., 3.8209397e-04,
        7.9809601e-05, 4.6917489e-03],
       [4.2757654e-04, 3.6888971e-04, 1.3049397e-03, ..., 4.3699294e-04,
        9.4055176e-05, 7.3122135e-03],
       [9.4765233e-04, 1.9827638e-04, 1.4930047e-03, ..., 2.6103374e-04,
        4.8726488e-05, 1.9566666e-03],
       ...,
       [3.5708238e-04, 3.2209119e-04, 1.4421352e-03, ..., 2.7774327e-04,
        6.4025066e-05, 4.9938834e-03],
       [1.9607815e-04, 3.5304183e-04, 6.3665729e-04, ..., 2.7991409e-04,
        5.7735673e-05, 4.4927429e-03],
       [1.9809089e-04, 3.5531036e-04, 6.5199059e-04, ..., 2.9911712e-04,
        6.2356026e-05, 4.5604524e-03]], dtype=float32)

In [59]:
numbers = []
for pred in prediction:
  formatted_numbers = [format(num, '.6f') for num in pred]
  numbers.append(formatted_numbers)

In [63]:
print(max(numbers[0]))

0.413828


In [ ]:
#Terminate
import os, signal
os.kill(os.getpid(), signal.SIGKILL)